## For YouTube Channels

In [28]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
import csv
import re

In [29]:
api_key = 'Enter your api key'
# 'UCVJU_IChPMOe8RWkdVQjtfQ'
channel_ids = []
st = input('Enter the channel id')
channel_ids.append(st)
youtube = build('youtube', 'v3', developerKey=api_key)

In [30]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                    Description = response['items'][i]['snippet']['description'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                    )
        all_data.append(data)
    
    return all_data

In [31]:
channel_statistics = get_channel_stats(youtube, channel_ids)

In [32]:
channel_data = pd.DataFrame(channel_statistics)

In [33]:
# Change channel name here
channel_name = channel_statistics[0]['Channel_name']
playlist_id = channel_data.loc[channel_data['Channel_name']==channel_name, 'playlist_id'].iloc[0]

In [34]:
def get_video_ids(youtube, playlist_id):
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [35]:
def get_video_details(youtube, video_ids):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute()
        
        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Description = video['snippet']['description']
                               )
            all_video_stats.append(video_stats)
    
    return all_video_stats

In [36]:
video_ids = get_video_ids(youtube, playlist_id)
print(len(video_ids))

1181


In [37]:
video_details = get_video_details(youtube, video_ids)

In [38]:
def remove_emoji(text):
    from cleantext import clean
    text = clean(text, no_emoji=True)
    return text
    
def remove_punc(test_str):
    punc = '''~!@#$%^&*()_+`-=[]{}|\;':",./<>?1234567890'''
    for ele in test_str:
        if ele in punc:
            test_str = test_str.replace(ele, "")
    return test_str

In [39]:
def keyword(data):
    word = data.split()
    dict = {}
    for j in word:
        if j in dict.keys():
            dict[j] += 1
        else:
            dict[j] = 1
    return dict

In [40]:
yes = 0
no = 0
for i in range (len(video_details)):
# importing info of a single video to a dictionary
    video_data = str(video_details[i])
# Removing Links from text
    video_data = re.sub('http://\S+|https://\S+', ' ', video_data)
    vid = re.sub('http[s]?://\S+', ' ', video_data)
    video_data = re.sub(r"http\S+", " ", video_data)
# removing puncuations and emojis from the text
    video_data = remove_punc(video_data)
    video_data = remove_emoji(video_data)
# finding keywords
    video_key = keyword(video_data)
# copying the non educational keywords to a dictionary
    with open('non_edu.csv', mode='r', encoding = "utf8") as ne:
        reader = csv.reader(ne)
        non_edu = {rows[0]:rows[1] for rows in reader}
# deleting the non educational words from keyword
    cd = []
    non_edu_dict = {}
    for j in non_edu:
        if j in video_key.keys():
            cd.append(j)
    for k in cd:
        non_edu_dict[k] = video_key[k]
        del video_key[k]
# copying the educational keywords to a dictionary
    with open('keyword_12.csv', mode='r', encoding = "utf8") as f:
        reader = csv.reader(f)
        key_dict = {rows[0]:rows[1] for rows in reader}
# Comparing my dataset and video keyword
    i = 0
    for key in video_key:
        if key in key_dict:
            i = i+1
    
    per = i*100/(len(video_key))
    # print(str(per)+'%')
    
# Counting true and false
    if per >30:
        yes = yes+1
    else:
        no=no+1

print('Total no of videos: '+ str(len(video_ids)))
print('No of Educational Videos: '+ str(yes))
print('No of Non-Educational Videos: '+ str(no))


Total no of videos: 1181
No of Educational Videos: 1087
No of Non-Educational Videos: 94
